<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/PredireReussite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prédire la réussite des études

Dans cet exercice, il s'agit d'étudier un ensemble de données sur la performance d'étudiantes et d'étudiants aux hautes études en fonction du contexte local, familliale et personnel.

Le but est de créer un réseau de neurones capable de prédire la réussite d'une personne en fonction de ces données.

Le jeux de données (dataset) est issu du site [kaggle](https://www.kaggle.com/); il comporte 145 exemples, composés de 20 critères, et d'une sortie (le niveau de réussite).

Voici le fichier à télécharger : [student_prediction_td.csv](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/student_prediction_td.csv)

---
Les données représentant les personnes sont les suivantes : 
 - **Student Age** (1: 18-21, 2: 22-25, 3: above 26)
 - **Sex** (1: female, 2: male)
 - **Graduated high-school type:** (1: private, 2: state, 3: other)
 - **Scholarship type:** (1: None, 2: 25%, 3: 50%, 4: 75%, 5: Full)
 - **Additional work:** (1: Yes, 2: No)
 - **Regular artistic or sports activity:** (1: Yes, 2: No)
 - **Do you have a partner:** (1: Yes, 2: No)
 - **Transportation to the university:** (1: Bus, 2: Private car/taxi, 3: bicycle, 4: Other)
 - **Accommodation type in Cyprus:** (1: rental, 2: dormitory, 3: with family, 4: Other)
 - **Mother's education:** (1: primary school, 2: secondary school, 3: high school, 4: university, 5: MSc., 6: Ph.D.)
 - **Father's education:** (1: primary school, 2: secondary school, 3: high school, 4: university, 5: MSc., 6: Ph.D.)
 - **Number of sisters/brothers (if available):** (1: 1, 2:, 2, 3: 3, 4: 4, 5: 5 or above)
 - **Parental status:** (1: married, 2: divorced, 3: died - one of them or both)
 - **Weekly study hours:** (1: None, 2: <5 hours, 3: 6-10 hours, 4: 11-20 hours, 5: more than 20 hours)
 - **Reading frequency (non-scientific books/journals):** (1: None, 2: Sometimes, 3: Often)
 - **Attendance to classes** (1: always, 2: sometimes, 3: never)
 - **Preparation to midterm exams 1:** (1: alone, 2: with friends, 3: not applicable)
 - **Preparation to midterm exams 2:** (1: closest date to the exam, 2: regularly during the semester, 3: never)
 - **Taking notes in classes:** (1: never, 2: sometimes, 3: always)
 - **Listening in classes:** (1: never, 2: sometimes, 3: always)
 - **Discussion improves my interest and success in the course:** (1: never, 2: sometimes, 3: always)

La sortie est codée ainsi : 
 - **OUTPUT Grade** (0: Fail, 1: DD, 2: DC, 3: CC, 4: CB, 5: BB, 6: BA, 7: AA)

---
## Apprentissage par l'exemple

Basez vous sur l'exercice/l'exemple de détection d'alertes, ou de classification de mail pour coder par vous même le réseau.

---
## Importer les librairies

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
#pour les graphiques
import matplotlib.pyplot as plt
#SI bug plus loin lors du dessin des graphiques, ajouter ces 2 lignes
#import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Définir les entrées et sorties attendues

Les entrées et sorties correspondent valeurs du tableau csv, normalisées sur [0,1].

Ici, les entrées d'apprentissage sont constituées de 70% des exemples. 

Les entrées et sorties de validations sont donc constitués des 30% restants

In [18]:
#transformer les données en entrées et sorties
# attention, il vaut mieux rester entre 0 & 1, mais surtout pour la sortie !

original = np.array([
[2, 2, 3, 3, 1, 2, 2, 1, 1, 1, 2, 3, 1, 3, 2, 1, 1, 1, 3, 2, 1], 
[2, 2, 3, 3, 1, 2, 2, 1, 1, 2, 3, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1], 
[2, 2, 2, 3, 2, 2, 2, 4, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1], 
[1, 1, 1, 3, 1, 2, 1, 1, 2, 1, 2, 5, 1, 3, 1, 1, 1, 2, 3, 2, 1], 
[2, 2, 1, 3, 2, 2, 1, 1, 4, 3, 3, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1], 
[2, 2, 2, 3, 2, 2, 2, 1, 1, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2], 
[1, 2, 2, 4, 2, 2, 2, 1, 3, 1, 3, 1, 1, 2, 2, 2, 1, 1, 3, 3, 5], 
[1, 1, 2, 3, 1, 1, 1, 2, 3, 4, 3, 1, 1, 1, 2, 1, 3, 1, 3, 2, 2], 
[2, 1, 3, 3, 2, 1, 1, 1, 3, 2, 4, 2, 1, 1, 2, 1, 1, 1, 3, 2, 5], 
[2, 1, 2, 3, 2, 2, 1, 4, 2, 1, 2, 3, 1, 2, 2, 2, 1, 1, 2, 2, 0], 
[1, 1, 1, 3, 2, 2, 2, 2, 3, 3, 4, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2], 
[1, 1, 1, 4, 1, 1, 2, 2, 3, 5, 5, 1, 1, 3, 3, 1, 3, 2, 3, 1, 0], 
[1, 1, 1, 4, 2, 2, 2, 1, 1, 3, 5, 4, 2, 3, 2, 1, 1, 1, 2, 2, 0], 
[2, 1, 2, 5, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 3, 2, 1], 
[3, 2, 2, 4, 1, 1, 2, 4, 2, 3, 1, 2, 1, 2, 2, 1, 1, 1, 2, 3, 2], 
[2, 2, 2, 3, 2, 2, 2, 1, 2, 4, 4, 2, 1, 2, 2, 2, 1, 1, 3, 2, 2], 
[1, 1, 2, 5, 2, 1, 2, 1, 1, 2, 2, 4, 1, 2, 2, 2, 1, 1, 3, 2, 1], 
[2, 2, 2, 3, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2], 
[1, 1, 2, 4, 2, 2, 2, 1, 1, 2, 2, 5, 1, 5, 3, 2, 1, 1, 3, 1, 2], 
[1, 2, 1, 3, 2, 2, 1, 2, 2, 3, 3, 3, 1, 4, 2, 1, 1, 1, 3, 2, 3], 
[1, 2, 2, 5, 1, 2, 1, 4, 2, 3, 3, 3, 2, 4, 2, 1, 1, 2, 3, 1, 1], 
[1, 2, 2, 5, 2, 2, 1, 4, 2, 2, 2, 4, 1, 3, 2, 1, 1, 1, 3, 1, 1], 
[2, 2, 2, 3, 1, 2, 1, 1, 1, 1, 1, 4, 2, 3, 3, 1, 1, 2, 3, 1, 3], 
[3, 2, 2, 2, 1, 1, 2, 1, 1, 1, 4, 3, 1, 3, 2, 2, 1, 1, 3, 2, 1], 
[2, 2, 2, 3, 2, 2, 2, 1, 1, 3, 1, 3, 1, 1, 2, 1, 1, 1, 2, 1, 2], 
[2, 2, 2, 3, 2, 2, 1, 1, 2, 1, 4, 3, 1, 2, 2, 1, 1, 1, 2, 1, 3], 
[2, 2, 2, 3, 2, 1, 1, 1, 1, 4, 4, 4, 1, 2, 3, 1, 1, 1, 3, 3, 1], 
[1, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 4, 1, 3, 2, 2, 1, 1, 3, 1, 1], 
[3, 2, 2, 3, 2, 2, 1, 4, 2, 2, 2, 4, 1, 2, 2, 1, 1, 1, 3, 2, 3], 
[2, 2, 3, 4, 2, 2, 2, 4, 2, 3, 3, 2, 1, 2, 2, 1, 1, 1, 3, 2, 5], 
[2, 2, 2, 5, 1, 1, 1, 1, 2, 1, 1, 5, 1, 2, 2, 1, 2, 1, 2, 3, 5], 
[3, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 4, 1, 2, 2, 1, 1, 1, 3, 3, 3], 
[2, 1, 2, 3, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1], 
[2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 2, 5, 1, 2, 1, 1, 1, 1, 1, 3, 2], 
[1, 2, 1, 3, 2, 2, 1, 1, 1, 3, 3, 3, 1, 2, 1, 2, 1, 1, 2, 2, 2], 
[1, 2, 1, 4, 2, 2, 2, 1, 1, 1, 2, 5, 1, 2, 2, 2, 1, 1, 2, 1, 1], 
[2, 2, 3, 4, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2], 
[2, 2, 2, 3, 1, 1, 1, 2, 3, 3, 3, 1, 1, 2, 2, 1, 1, 1, 3, 2, 1], 
[2, 2, 2, 5, 2, 2, 2, 1, 1, 3, 3, 1, 1, 3, 2, 1, 2, 1, 2, 2, 2], 
[2, 1, 2, 3, 2, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1], 
[1, 2, 1, 3, 2, 2, 2, 1, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1], 
[3, 2, 2, 3, 1, 2, 2, 1, 2, 1, 4, 2, 1, 3, 2, 2, 1, 1, 3, 2, 1], 
[2, 2, 2, 3, 2, 1, 2, 4, 2, 4, 4, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1], 
[1, 2, 2, 3, 2, 2, 1, 1, 1, 2, 3, 3, 1, 2, 2, 2, 1, 1, 3, 1, 4], 
[2, 2, 3, 3, 2, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 2, 1, 3, 2, 1], 
[1, 2, 2, 3, 2, 2, 1, 1, 1, 2, 3, 5, 1, 3, 2, 1, 1, 1, 2, 2, 3], 
[2, 2, 2, 3, 2, 2, 1, 1, 1, 1, 2, 3, 1, 2, 2, 1, 1, 1, 2, 2, 5], 
[2, 2, 2, 3, 2, 2, 1, 1, 2, 4, 3, 1, 1, 2, 2, 2, 1, 1, 2, 1, 3], 
[1, 2, 2, 3, 2, 1, 1, 1, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 3, 2, 1], 
[1, 2, 1, 4, 2, 2, 2, 4, 1, 2, 3, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2], 
[2, 2, 2, 3, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1], 
[2, 1, 3, 3, 1, 1, 2, 1, 1, 1, 2, 5, 1, 1, 2, 2, 1, 1, 2, 3, 4], 
[2, 1, 2, 3, 1, 2, 2, 1, 1, 1, 3, 4, 1, 3, 1, 1, 1, 2, 2, 2, 1], 
[2, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 5, 1, 1, 2, 2, 1, 1, 2, 2, 5], 
[2, 2, 2, 3, 2, 2, 2, 4, 2, 1, 4, 2, 1, 2, 1, 1, 1, 2, 3, 1, 3], 
[3, 2, 2, 3, 1, 2, 1, 1, 2, 1, 1, 5, 3, 1, 2, 1, 3, 3, 1, 3, 3], 
[2, 2, 2, 3, 2, 1, 2, 1, 1, 1, 3, 5, 1, 4, 2, 1, 1, 2, 3, 2, 5], 
[2, 2, 2, 3, 1, 1, 2, 1, 1, 4, 2, 3, 1, 2, 3, 1, 1, 1, 3, 2, 4], 
[3, 2, 2, 3, 2, 2, 1, 1, 1, 1, 3, 5, 2, 3, 2, 1, 1, 1, 3, 2, 3], 
[2, 2, 2, 3, 2, 1, 1, 1, 1, 4, 4, 3, 1, 3, 2, 1, 1, 1, 3, 3, 5], 
[2, 1, 2, 3, 2, 2, 2, 2, 1, 6, 1, 3, 1, 1, 2, 1, 1, 1, 1, 3, 2], 
[1, 2, 3, 3, 2, 1, 2, 1, 2, 1, 3, 4, 1, 3, 1, 2, 1, 1, 3, 2, 5], 
[2, 2, 2, 3, 2, 2, 2, 4, 2, 2, 3, 2, 1, 3, 2, 1, 1, 1, 2, 3, 3], 
[2, 2, 2, 4, 2, 2, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 2, 2, 5], 
[2, 2, 3, 5, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 3, 2, 3], 
[1, 2, 2, 3, 2, 2, 1, 1, 1, 2, 4, 3, 1, 1, 1, 2, 2, 1, 2, 1, 2], 
[2, 2, 2, 3, 2, 2, 1, 1, 1, 3, 2, 5, 1, 2, 2, 1, 1, 1, 3, 2, 5], 
[2, 2, 3, 3, 1, 1, 2, 1, 2, 3, 4, 4, 2, 2, 2, 2, 1, 1, 2, 2, 1], 
[2, 1, 2, 4, 1, 2, 2, 1, 1, 2, 3, 5, 3, 1, 2, 2, 1, 1, 2, 2, 5], 
[2, 1, 2, 4, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 3, 2, 5], 
[1, 2, 2, 4, 2, 1, 1, 1, 1, 4, 3, 1, 2, 2, 2, 1, 1, 1, 2, 2, 7], 
[1, 1, 3, 4, 2, 2, 2, 1, 3, 1, 3, 5, 1, 2, 2, 1, 1, 1, 2, 3, 6], 
[1, 2, 2, 3, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 3, 1, 1, 1, 2, 2, 6], 
[2, 2, 2, 4, 2, 2, 2, 1, 2, 1, 3, 4, 1, 3, 1, 1, 1, 2, 3, 2, 6], 
[1, 2, 2, 4, 2, 2, 2, 1, 1, 2, 2, 3, 1, 2, 2, 1, 2, 1, 3, 2, 7], 
[1, 2, 2, 4, 2, 1, 2, 1, 3, 1, 5, 4, 1, 3, 3, 1, 1, 1, 2, 2, 7], 
[2, 2, 1, 2, 2, 1, 2, 1, 1, 5, 4, 3, 1, 2, 3, 1, 3, 2, 2, 1, 4], 
[1, 2, 1, 2, 2, 2, 1, 2, 2, 6, 5, 2, 1, 2, 2, 2, 3, 1, 2, 2, 7], 
[2, 1, 2, 4, 1, 1, 2, 1, 1, 1, 3, 4, 1, 2, 3, 2, 1, 1, 3, 3, 4], 
[2, 2, 2, 4, 2, 2, 2, 1, 1, 1, 3, 4, 1, 1, 2, 1, 1, 1, 3, 3, 3], 
[2, 1, 2, 3, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 3, 4], 
[3, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 4, 1, 2, 2, 1, 1, 1, 3, 3, 3], 
[2, 2, 2, 4, 1, 2, 1, 1, 1, 3, 1, 3, 1, 2, 2, 1, 1, 1, 3, 3, 7], 
[2, 2, 3, 3, 2, 2, 2, 1, 1, 4, 4, 2, 1, 1, 2, 1, 3, 1, 3, 2, 7], 
[3, 2, 3, 3, 1, 2, 1, 1, 2, 4, 2, 5, 3, 1, 2, 1, 3, 3, 3, 3, 7], 
[1, 2, 2, 5, 2, 2, 2, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 3, 2, 4], 
[2, 2, 2, 4, 2, 2, 2, 4, 2, 3, 3, 2, 1, 2, 2, 1, 1, 1, 3, 2, 5], 
[2, 2, 2, 3, 2, 1, 2, 1, 1, 3, 4, 2, 1, 2, 2, 1, 2, 1, 2, 2, 6], 
[1, 2, 2, 4, 2, 1, 1, 1, 1, 4, 4, 1, 1, 2, 2, 1, 1, 2, 3, 1, 6], 
[2, 2, 2, 3, 2, 2, 2, 1, 1, 3, 5, 3, 1, 2, 2, 2, 1, 1, 3, 1, 6], 
[2, 1, 2, 3, 2, 1, 1, 2, 3, 3, 3, 1, 1, 2, 2, 1, 1, 1, 2, 3, 6], 
[2, 2, 2, 5, 1, 1, 1, 1, 2, 4, 1, 5, 1, 2, 2, 1, 1, 1, 2, 3, 6], 
[1, 2, 2, 3, 2, 2, 2, 1, 1, 3, 4, 4, 1, 3, 2, 1, 1, 1, 3, 2, 7], 
[1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 4, 1, 5, 2, 1, 1, 1, 3, 3, 4], 
[2, 2, 2, 3, 2, 2, 1, 1, 1, 3, 2, 5, 1, 2, 2, 1, 1, 1, 3, 2, 6], 
[1, 2, 3, 5, 1, 1, 1, 2, 3, 3, 4, 1, 1, 2, 2, 1, 1, 1, 3, 2, 5], 
[1, 2, 2, 4, 2, 1, 1, 2, 3, 2, 2, 3, 1, 2, 3, 1, 1, 1, 3, 2, 7], 
[1, 2, 2, 4, 1, 2, 2, 1, 3, 3, 3, 4, 1, 2, 2, 1, 1, 1, 3, 2, 6], 
[1, 2, 2, 4, 2, 1, 2, 4, 2, 2, 4, 2, 1, 2, 3, 1, 1, 1, 3, 1, 7], 
[2, 2, 2, 3, 2, 2, 2, 4, 2, 3, 4, 2, 1, 3, 2, 1, 1, 1, 3, 2, 7], 
[1, 2, 2, 4, 2, 2, 2, 2, 3, 4, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 6], 
[1, 2, 2, 4, 2, 2, 1, 1, 3, 2, 3, 1, 1, 2, 2, 1, 1, 1, 3, 3, 7], 
[1, 2, 2, 3, 2, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 1, 1, 3, 1, 7], 
[1, 2, 1, 4, 2, 2, 1, 1, 2, 1, 3, 4, 1, 2, 3, 1, 1, 1, 2, 2, 7], 
[1, 1, 2, 3, 2, 2, 2, 2, 2, 4, 4, 1, 1, 2, 2, 1, 1, 1, 2, 3, 3], 
[1, 2, 2, 4, 1, 1, 2, 1, 1, 2, 2, 4, 1, 2, 3, 2, 1, 2, 2, 1, 7], 
[1, 2, 2, 4, 2, 1, 2, 1, 2, 2, 2, 1, 1, 3, 2, 1, 3, 2, 2, 2, 7], 
[1, 1, 2, 4, 1, 1, 2, 1, 3, 1, 3, 5, 1, 2, 2, 1, 2, 1, 2, 3, 6], 
[2, 1, 1, 5, 2, 1, 2, 2, 1, 1, 2, 1, 3, 3, 3, 2, 1, 1, 2, 3, 6], 
[1, 2, 1, 3, 2, 1, 1, 2, 2, 4, 4, 2, 3, 3, 2, 1, 2, 2, 3, 2, 7], 
[2, 2, 2, 3, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2], 
[1, 1, 1, 5, 2, 1, 2, 1, 2, 3, 3, 1, 1, 1, 2, 1, 1, 1, 3, 1, 2], 
[2, 1, 3, 3, 1, 2, 2, 3, 3, 1, 1, 2, 1, 1, 1, 1, 2, 1, 3, 3, 2], 
[2, 1, 3, 3, 2, 2, 2, 4, 2, 4, 4, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1], 
[2, 1, 2, 5, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2], 
[2, 1, 2, 5, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 3, 1, 2, 2, 3, 2, 1], 
[2, 1, 2, 5, 2, 2, 2, 1, 1, 1, 1, 5, 1, 3, 3, 1, 1, 1, 3, 3, 1], 
[3, 1, 1, 3, 2, 1, 2, 1, 2, 4, 4, 1, 1, 2, 2, 2, 1, 1, 2, 3, 1], 
[1, 2, 2, 5, 2, 1, 1, 4, 2, 1, 2, 3, 1, 1, 2, 1, 3, 1, 3, 3, 1], 
[2, 1, 2, 4, 2, 1, 2, 1, 2, 3, 3, 2, 3, 5, 2, 1, 1, 2, 2, 2, 2], 
[2, 1, 1, 3, 1, 1, 1, 2, 3, 3, 3, 2, 1, 1, 1, 2, 2, 1, 3, 3, 1], 
[2, 1, 2, 3, 1, 1, 1, 2, 1, 1, 1, 5, 3, 5, 3, 1, 2, 1, 1, 1, 0], 
[1, 2, 2, 5, 2, 1, 2, 1, 1, 3, 3, 2, 1, 2, 1, 1, 2, 1, 3, 2, 2], 
[2, 1, 3, 3, 2, 2, 2, 1, 1, 3, 2, 4, 1, 2, 2, 1, 1, 1, 3, 3, 1], 
[1, 1, 2, 4, 1, 1, 1, 1, 3, 2, 3, 2, 1, 4, 2, 1, 2, 2, 3, 3, 3], 
[1, 1, 2, 5, 1, 1, 2, 1, 3, 1, 1, 5, 1, 3, 2, 1, 1, 2, 3, 2, 2], 
[1, 1, 1, 4, 1, 1, 1, 2, 3, 4, 6, 2, 1, 4, 2, 1, 2, 1, 3, 3, 3], 
[1, 1, 2, 4, 2, 2, 2, 4, 3, 3, 2, 2, 1, 2, 1, 1, 1, 1, 3, 2, 1], 
[1, 1, 2, 4, 2, 1, 1, 4, 3, 3, 4, 2, 1, 2, 1, 1, 1, 1, 3, 2, 0], 
[1, 1, 2, 3, 1, 1, 2, 1, 1, 1, 1, 3, 1, 4, 2, 1, 2, 2, 3, 2, 3], 
[1, 1, 2, 3, 1, 1, 2, 1, 1, 1, 2, 3, 1, 4, 2, 1, 1, 2, 3, 1, 1], 
[1, 1, 1, 5, 2, 1, 2, 2, 2, 4, 3, 1, 1, 4, 1, 1, 1, 2, 3, 2, 4], 
[1, 1, 1, 5, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 2, 1, 2, 1, 3, 2, 3], 
[1, 1, 2, 5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3, 3], 
[1, 1, 2, 4, 2, 1, 2, 2, 3, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1], 
[2, 1, 2, 3, 1, 1, 2, 4, 2, 3, 3, 1, 1, 2, 3, 1, 1, 1, 3, 3, 2], 
[1, 1, 2, 3, 1, 1, 1, 2, 3, 3, 3, 3, 1, 2, 2, 1, 3, 1, 3, 2, 0], 
[1, 1, 1, 5, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 1, 2], 
[1, 1, 2, 4, 1, 1, 1, 2, 3, 3, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 0], 
[1, 1, 2, 4, 1, 2, 1, 2, 3, 3, 3, 1, 1, 1, 1, 2, 2, 1, 2, 3, 0], 
[2, 1, 2, 3, 1, 1, 2, 1, 2, 1, 2, 2, 2, 3, 3, 1, 1, 1, 2, 1, 5], 
[1, 1, 2, 4, 2, 2, 2, 4, 2, 1, 1, 5, 1, 3, 2, 2, 1, 1, 3, 2, 5], 
[1, 1, 1, 4, 2, 2, 2, 1, 1, 3, 4, 4, 1, 2, 2, 1, 1, 1, 3, 3, 1], 
[2, 1, 2, 4, 1, 1, 1, 2, 3, 4, 4, 1, 1, 2, 2, 1, 2, 1, 2, 1, 4], 
[1, 1, 1, 5, 2, 2, 2, 1, 1, 3, 1, 5, 1, 3, 1, 1, 2, 1, 3, 2, 3]]
                   , float)
nb_lignes = original.shape[0]
nb_val = original.shape[1]-1

##on fait debuter les valeurs à 0..
uns = np.ones(nb_val+1)
tableau = original - uns

maxis = np.max(original, axis=0)
maxis[nb_val] = 1
## on ramène tout à une valeur entre 0 et 1 sauf la derniere ligne ?
### pas forcement une bonne solution ?
### on tente !
tableau = original/maxis

In [ ]:
#on prend un pourcentage de lignes pour apprendre, le reste pour valider
#il faut mélanger les lignes pour pouvoir avoir des exemples d'apprentissage
# et de validation homogènes (autrement, les exemples de tests seraient tous des exemples négatifs (non dangereux))
#np.random.shuffle(tableau)

#on décide que 90% des lignes sont des lignes d'apprentissage
nb_lignes_app = nb_lignes * 90 // 100
nb_lignes_val = nb_lignes - nb_lignes_app

# les premieres colonnes sont les entrees 
# la derniere colonne est la sortie

#on remplit les entrees et sorties d'apprentissage
entrees_app = tableau[0:nb_lignes_app, 0:nb_val]
sorties_app = tableau[0:nb_lignes_app, nb_val:nb_val+1]

#on remplit les entrees et sorties de validation
entrees_val = tableau[nb_lignes_app:nb_lignes, 0:nb_val]
sorties_val = tableau[nb_lignes_app:nb_lignes, nb_val:nb_val+1]

### Construire le réseau et tester.....
Reprenez les codes de SolutionDetectionAlertes et adaptez les

---
## Utilisation du réseau
Déterminez la probabilité de réussite :
 1. d'une femme de 25 ans, étudiant à plein temps dans une université, n'ayant pas d'emploi, et membre d'un groupe de musique, en couple dans une location, se rendant aux cours en bus, dont les parents se sont arrêtés au lycée, ayant 1 frère, qui assiste aux 20h de cours hebdomadaires, qui révise seule et à la dernière minute, mais qui prend des notes et écoute en classe ?
 2. de votre cas personnel ?
 
- Quel est l'impact du niveau d'étude des parents sur la réussite d'une personne ?

- Quel est l'impact de l'écoute et de la prise de notes ?